In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade praw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 4.9 MB/s eta 0:00:00


In [ ]:
import praw
import pandas as pd
import os

In [ ]:
# Reddit API credentials
client_id = 'ID'
client_secret = 'SECRET'
user_agent = '''EXPLANATION'''

# Initialize Reddit instance
reddit = praw.Reddit(client_id=client_id,
                     client_secret=client_secret,
                     user_agent=user_agent,
                     check_for_async=False)

In [ ]:
# List of Reddit usernames
# This is performed after saving usernames that were already located via flairs
usernames = users_list

# Define the path to save the CSV file in Google Drive
csv_path = 'path'

# Initialize the CSV file with headers if it doesn't exist
if not os.path.exists(csv_path):
    df = pd.DataFrame(columns=['username', 'type', 'post_id', 'created_utc', 'title', 'body', 'subreddit'])
    df.to_csv(csv_path, index=False)

# Function to append data to CSV
def append_to_csv(data, path):
    df = pd.DataFrame([data])
    df.to_csv(path, mode='a', header=False, index=False)

for username in usernames:
    print(f'Processing user {username}')
    try:
        user = reddit.redditor(username)

        # Fetch posts (submissions)
        for submission in user.submissions.new(limit=None):
            data = {
                'username': username,
                'type': 'post',
                'post_id': submission.id,
                'created_utc': submission.created_utc,
                'title': submission.title,
                'body': submission.selftext,
                'subreddit': submission.subreddit.display_name
            }
            append_to_csv(data, csv_path)

        # Fetch comments
        for comment in user.comments.new(limit=None):
            data = {
                'username': username,
                'type': 'comment',
                'post_id': comment.id,
                'created_utc': comment.created_utc,
                'title': None,  # No title for comments
                'body': comment.body,
                'subreddit': comment.subreddit.display_name
            }
            append_to_csv(data, csv_path)

    except Exception as e:
        print(f"Error processing {username}: {e}")

print(f"Data saved to {csv_path}")

In [ ]:
import re

# List of common phrases
phrases = [
    r"i have",
    r"i've",
    r"ive",
    r"i'm",
    r"suffering from",
    r"suffer from",
    r"being",
    r"diagnosis of",
    r"diagnosed with",
    r"diagnosed me with",
    r"diagnostic of",
    r"diagnosed",
    r"hospitalized with",
    r"hospital with",
    r"my"
]

# List of mental disorders
disorders = [
    "schzo",
    "schizopheni",
    "scizophr",
    "schophreni",
    "schizopr",
    "shizophrenia",
    "schizphrenia",
    "schzophrenia",
    "schizoprenia",
    "scizophrenia",
    "schizohrenia",
    "schzophreni",
    "schzophrnia",
    "schizohrnia",
    "schizophenia",
    "schizorphenia",
    "schizoprhenia",
    "schzophrena",
    "schizorena",
    "scizophrnia"
]

patterns = [fr"{phrase}\s*({disorder})" for phrase in phrases for disorder in disorders]

compiled_patterns = [re.compile(pattern, re.IGNORECASE) for pattern in patterns]
print(compiled_patterns)


[re.compile('i have\\s*(schzo)', re.IGNORECASE), re.compile('i have\\s*(schizopheni)', re.IGNORECASE), re.compile('i have\\s*(scizophr)', re.IGNORECASE), re.compile('i have\\s*(schophreni)', re.IGNORECASE), re.compile('i have\\s*(schizopr)', re.IGNORECASE), re.compile('i have\\s*(shizophrenia)', re.IGNORECASE), re.compile('i have\\s*(schizphrenia)', re.IGNORECASE), re.compile('i have\\s*(schzophrenia)', re.IGNORECASE), re.compile('i have\\s*(schizoprenia)', re.IGNORECASE), re.compile('i have\\s*(scizophrenia)', re.IGNORECASE), re.compile('i have\\s*(schizohrenia)', re.IGNORECASE), re.compile('i have\\s*(schzophreni)', re.IGNORECASE), re.compile('i have\\s*(schzophrnia)', re.IGNORECASE), re.compile('i have\\s*(schizohrnia)', re.IGNORECASE), re.compile('i have\\s*(schizophenia)', re.IGNORECASE), re.compile('i have\\s*(schizorphenia)', re.IGNORECASE), re.compile('i have\\s*(schizoprhenia)', re.IGNORECASE), re.compile('i have\\s*(schzophrena)', re.IGNORECASE), re.compile('i have\\s*(schizo

In [ ]:
example_texts = [
    "I have been diagnosed with schizophrenia recently, and it's been tough.",
    "I'm currently suffering from schizophrenic episodes.",
    "The doctor diagnosed me with schizo last year.",
    "I've been feeling symptoms of SZ for a while now.",
    "ive been suffering from something like schzo but not sure.",
    "My diagnosis of schizophrenia was confirmed by two doctors.",
    "Being diagnosed with SZ has changed my life completely.",
    "Suffer from schizophrenic symptoms, but I'm trying to manage.",
    "I have schizophrenia, but I try to live a normal life."
]

# Check each text for matches
for text in example_texts:
    for pattern in compiled_patterns:
        match = pattern.search(text.lower())
        if match:
            print(f"Match found: '{match.group()}' in text: '{text}'")
            break

Match found: 'diagnosed with schizophrenia' in text: 'I have been diagnosed with schizophrenia recently, and it's been tough.'
Match found: 'suffering from schizophrenic' in text: 'I'm currently suffering from schizophrenic episodes.'
Match found: 'diagnosed me with schizo' in text: 'The doctor diagnosed me with schizo last year.'
Match found: 'diagnosis of schizophrenia' in text: 'My diagnosis of schizophrenia was confirmed by two doctors.'
Match found: 'diagnosed with sz' in text: 'Being diagnosed with SZ has changed my life completely.'
Match found: 'suffer from schizophrenic' in text: 'Suffer from schizophrenic symptoms, but I'm trying to manage.'
Match found: 'i have schizophrenia' in text: 'I have schizophrenia, but I try to live a normal life.'


In [ ]:
import re
import pandas as pd
import csv

# Read the input CSV file from Google Drive
input_csv_path = 'path'
df = pd.read_csv(input_csv_path)

# Path to the output CSV file where matches will be saved
output_csv_path = 'path'

# Open the output CSV file in append mode
with open(output_csv_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Write the header to the output CSV
    header = list(df.columns) + ['matched_patterns']
    writer.writerow(header)

    # Iterate through each row, checking the content in columns E and F (index 4 and 5)
    for index, row in df.iterrows():
        matches = []
        title = str(row['title']).lower() if pd.notna(row['title']) else ''
        body = str(row['body']).lower() if pd.notna(row['body']) else ''

        for pattern in compiled_patterns:
            # Check if the pattern is found in either the lowercase title or body
            if pattern.search(title) or pattern.search(body):
                matches.append(pattern.pattern)

        # If there were matches, save the row immediately to the output CSV
        if matches:
            matched_row = list(row) + ['; '.join(matches)]
            writer.writerow(matched_row)

print(f"Filtered data is done")

Filtered data is done


In [ ]:
import pandas as pd
from datetime import datetime
from google.colab import files


In [ ]:
uploaded = files.upload()

In [ ]:
#FOR SZ

input_csv = r"sz_found - sz_found (1).csv"
df = pd.read_csv(input_csv)


# Create a list to store usernames with "X" in the ninth column
# "X" means that self-disclosures were manually confirmed

usernames_with_X = df[df['include'] == 'X']['username'].tolist()

seen = set()
unique_list = []
for item in usernames_with_X:
    if item not in seen:
        unique_list.append(item)
        seen.add(item)

# Step 6: Convert 'created_utc' from Unix timestamp to readable date format
df_filtered = df[df['include'] == 'X'].copy()
df_filtered['created_utc'] = pd.to_datetime(df_filtered['created_utc'], unit='s').dt.date

# Step 7: Save the filtered data to a new CSV file in Google Drive
output_path = 'path'
df_filtered.to_csv(output_path, index=False)

# Step 8: Print the list of usernames with "X" in the ninth column
print("Usernames with 'X' in the ninth column:", unique_list)
print(len(unique_list))

In [ ]:
#FOR SZA

input_csv = r"sza_found - sza_found (1).csv"
df = pd.read_csv(input_csv)


# Create a list to store usernames with "X" in the ninth column
# "X" means that self-disclosures were manually confirmed
usernames_with_X = df[df['include'] == 'X']['username'].tolist()

seen = set()
unique_list = []
for item in usernames_with_X:
    if item not in seen:
        unique_list.append(item)
        seen.add(item)

# Step 6: Convert 'created_utc' from Unix timestamp to readable date format
df_filtered = df[df['include'] == 'X'].copy()
df_filtered['created_utc'] = pd.to_datetime(df_filtered['created_utc'], unit='s').dt.date

# Step 7: Save the filtered data to a new CSV file in Google Drive
output_path = 'path'
df_filtered.to_csv(output_path, index=False)

# Step 8: Print the list of usernames with "X" in the ninth column
print("Usernames with 'X' in the ninth column:", unique_list)
print(len(unique_list))

In [ ]:
#FOR SZA


input_csv = r"sza_found_filtered_data - sza_found_filtered_data.csv"
df = pd.read_csv(input_csv)


# Create a list to store usernames with "X" in the ninth column
usernames_with_X = df[df['include'] == 'X']['username'].tolist()

seen = set()
unique_list = []
for item in usernames_with_X:
    if item not in seen:
        unique_list.append(item)
        seen.add(item)

print('SZA:')
print(f'Before checking for duplicates: {len(usernames_with_X)}')
print(f'After checking for duplicates: {len(unique_list)}')
print(unique_list)





input_csv = r"sz_found_filtered_data - sz_found_filtered_data.csv"
df = pd.read_csv(input_csv)


# Create a list to store usernames with "X" in the ninth column
usernames_with_X = df[df['include'] == 'X']['username'].tolist()

seen = set()
unique_list = []
for item in usernames_with_X:
    if item not in seen:
        unique_list.append(item)
        seen.add(item)

print('SZ:')
print(f'Before checking for duplicates: {len(usernames_with_X)}')
print(f'After checking for duplicates: {len(unique_list)}')
print(unique_list)

In [ ]:
import csv

# Initialize an empty set to store the values
values_set = set()

# Open and read the CSV file
with open("path", mode='r', newline='', encoding='utf-8') as csvfile:
    csvreader = csv.reader(csvfile)

    # Skip the header row
    next(csvreader)

    # Iterate over each row in the CSV file
    for row in csvreader:
        # Add the value from the first column to the set
        values_set.add(row[0])

# Print the set to verify
print(values_set)
print(len(values_set))

In [ ]:
import re

# List of common phrases
phrases = [
    r"i have",
    r"i've",
    r"ive",
    r"i'm",
    r"suffering from",
    r"suffer from",
    r"being",
    r"diagnosis of",
    r"diagnosed with",
    r"diagnosed me with",
    r"diagnostic of",
    r"diagnosed",
    r"hospitalized with",
    r"hospital with",
    r"my"
]

# List of mental disorders
disorders = [
    # Intellectual Developmental Disorder (Intellectual Disability) V
    'intellectual disability', 'intellectual developmental', 'idd', 'mental retardation', 'developmental delay', 'developmental disabilit', 'hypertelorism', 'facies', 'snijders', 'child development deviation', 'development deviation', 'mental deficiency',

    # Language Disorder V
    'language', 'speech delay', 'expressive language', 'receptive language', 'aphasia', 'Acquired Language Disorder', 'language impairment', 'speech disorder', 'specific language disorder', 'sld', 'developmental disorder', 'language delay', 'semantic-pragmatic', 'semantic pragmatic', 'central auditory', 'auditory central', 'speech or language developmental',

    # Speech Sound Disorder V
    'speech sound', 'articulation', 'phonological', 'apraxia', 'dysarthria',

    # Social (Pragmatic) Communication Disorder V
    'social communication', 'pragmatic', 'social language',

    # Unspecified Communication Disorder V
    'communication', 'unspecified communication', 'speech and language', 'childhood communication', 'acquired communication', 'communicative', 'developmental communication', 'neurogenic', 'hearing disorder',

    # Autism Spectrum Disorder V
    'autism', 'asd', 'asperger', 'high-functioning autism', 'pervasive developmental disorder', 'autie', 'autistic', 'kenner', 'Cerebroatrophic Hyperammonemia', 'rett', 'akinetic',

    # Bipolar I V
    'bipolar', 'manic', 'mania',

    # Bipolar II V
    'hypomania',

    # Cyclothymia V
    'cyclothymi',

    # Factitious Disorder V
    'factitious', 'genser syndrome', 'pseudodementia', 'pseudopsychosis',

    # Alzheimer/Dementia V
    'alzheimer', 'dementia', 'cognitive decline', 'amentia', 'senile paranoid dementia', 'familial dementia', 'early-onset dementia', 'acute confuse senile', 'early onset alzheimer', 'familiar alzheimer disease', 'focal onset alzheimer', 'late onset alzheimer', 'presenile alzheimer', 'primary senile degenerative dementia', 'senile dementia',

    # Brain Injury V
    'brain injury', 'traumatic brain injury', 'tbi', 'concussion', 'head injury', 'focal brain inj', 'acute brain inj', 'brain laceration', 'chronic brain inj',

    # Lewy Body Disease V
    'lewy body disease', 'lewy body dementia', 'lbd', 'cortical lewy body disease', 'diffuse lewy body', 'lewy bod',

    # Frontotemporal Lobar Degeneration V
    'frontotemporal lobar degeneration', 'ftld', 'frontotemporal dementia', 'pick\'s disease', 'ddpac', 'familial pick\'s disease', 'hddd', 'semantic dementia', 'Wilhelmsen-Lynch Disease', 'Hereditary Dysphasic Disinhibition Dementia', 'Multiple System Tauopathy with Presenile Dementia',

    # Parkinson's Disease V
    'parkinson', 'parkinsonism', 'parkinsonian syndrome', 'idiopathic parkinson', 'paralysis agitans', 'primary parkinsonism',

    # Huntington's Disease V
    'huntington', 'hd', 'Akinetic-Rigid Variant of Huntington', 'chorea', 'Chronic Progressive Hereditary Chorea', 'Juvenile Huntington', 'Juvenile-Onset Huntington', 'Late-Onset Huntington Disease', 'Late Onset Huntington Disease', 'Juvenile Onset Huntington', 'Progressive Chorea',

    # Specific Learning Disorder (e.g., Dyslexia) V
    'specific learning', 'learning', 'dyslexia', 'dysgraphia', 'dyscalculia', 'sld',

    # Borderline Personality Disorder V
    'borderline', 'bpd', 'bp-d'
]

patterns = [fr"{phrase}\s*({disorder})" for phrase in phrases for disorder in disorders]

compiled_patterns = [re.compile(pattern, re.IGNORECASE) for pattern in patterns]
print(compiled_patterns)

[re.compile('i have\\s*(intellectual disability)', re.IGNORECASE), re.compile('i have\\s*(intellectual developmental)', re.IGNORECASE), re.compile('i have\\s*(idd)', re.IGNORECASE), re.compile('i have\\s*(mental retardation)', re.IGNORECASE), re.compile('i have\\s*(developmental delay)', re.IGNORECASE), re.compile('i have\\s*(developmental disabilit)', re.IGNORECASE), re.compile('i have\\s*(hypertelorism)', re.IGNORECASE), re.compile('i have\\s*(facies)', re.IGNORECASE), re.compile('i have\\s*(snijders)', re.IGNORECASE), re.compile('i have\\s*(child development deviation)', re.IGNORECASE), re.compile('i have\\s*(development deviation)', re.IGNORECASE), re.compile('i have\\s*(mental deficiency)', re.IGNORECASE), re.compile('i have\\s*(language)', re.IGNORECASE), re.compile('i have\\s*(speech delay)', re.IGNORECASE), re.compile('i have\\s*(expressive language)', re.IGNORECASE), re.compile('i have\\s*(receptive language)', re.IGNORECASE), re.compile('i have\\s*(aphasia)', re.IGNORECASE), 

In [ ]:
import re
import pandas as pd
import csv

# Read the input CSV file from Google Drive
input_csv_path = 'path'
df = pd.read_csv(input_csv_path)

# Path to the output CSV file where matches will be saved
output_csv_path = 'path'

# Open the output CSV file in append mode
with open(output_csv_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Write the header to the output CSV
    header = list(df.columns) + ['matched_patterns']
    writer.writerow(header)

    # Iterate through each row, checking the content in columns E and F (index 4 and 5)
    for index, row in df.iterrows():
        if row['username'] not in values_set:
          pass
        else:
          matches = []
          title = str(row['title']).lower() if pd.notna(row['title']) else ''
          body = str(row['body']).lower() if pd.notna(row['body']) else ''

          for pattern in compiled_patterns:
              # Check if the pattern is found in either the lowercase title or body
              if pattern.search(title) or pattern.search(body):
                  matches.append(pattern.pattern)

          # If there were matches, save the row immediately to the output CSV
          if matches:
              matched_row = list(row) + ['; '.join(matches)]
              writer.writerow(matched_row)

print(f"Filtered data is done")

Filtered data is done


In [ ]:
import csv
from datetime import datetime

# Open the input CSV file for reading and the output CSV file for writing
with open('path', 'r') as infile, open('path', 'w', newline='') as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile)

    # Write header to output file if the CSV contains a header
    header = next(reader)
    writer.writerow(header)  # Adjust this if there is no header

    # Iterate through each line in the CSV
    for row in reader:
        # Check if the value in the first column is in the list
        if row[0] in users_sz_final:
            # Convert the UTC timestamp (in the 4th column) to a readable date
            utc_timestamp = float(row[3])
            # Assuming the timestamp is in a typical format like '2024-09-28T14:45:30Z'
            actual_date = datetime.fromtimestamp(utc_timestamp)
            # Convert the date to a more readable format, e.g., 'YYYY-MM-DD HH:MM:SS'
            row[3] = actual_date.strftime("%Y-%m-%d %H:%M:%S")

            # Write the modified row to the new CSV file
            writer.writerow(row)

In [ ]:
import re
nationalities = [
    "Afghan", "Albanian", "Algerian", "American", "Andorran", "Angolan",
    "Antiguan", "Argentine", "Armenian", "Australian", "Austrian", "Azerbaijani",
    "Bahamian", "Bahraini", "Bangladeshi", "Barbadian", "Barbudan", "Batswana",
    "Belarusian", "Belgian", "Belizean", "Beninese", "Bhutanese", "Bolivian",
    "Bosnian", "Brazilian", "British", "Bruneian", "Bulgarian", "Burkinabé",
    "Burmese", "Burundian", "Cabo Verdean", "Cambodian", "Cameroonian", "Canadian",
    "Central African", "Chadian", "Chilean", "Chinese", "Colombian", "Comoran",
    "Congolese", "Costa Rican",
    "Croatian", "Cuban", "Cypriot", "Czech", "Danish", "Djiboutian", "Dominican",
    "Dutch", "East Timorese", "Ecuadorean", "Egyptian", "Emirati", "Equatorial Guinean",
    "Eritrean", "Estonian", "Ethiopian", "Fijian", "Finnish", "French",
    "Gabonese", "Gambian", "Georgian", "German", "Ghanaian", "Greek",
    "Grenadian", "Guatemalan", "Guinean", "Guinea-Bissauan", "Guyanese", "Haitian",
    "Honduran", "Hungarian", "Icelandic", "Indian", "Indonesian", "Iranian",
    "Iraqi", "Irish", "Israeli", "Italian", "Ivorian", "Jamaican",
    "Japanese", "Jordanian", "Kazakh", "Kenyan", "Kiribati", "Kittian and Nevisian",
    "Kuwaiti", "Kyrgyz", "Laotian", "Latvian", "Lebanese", "Liberian",
    "Libyan", "Liechtensteiner", "Lithuanian", "Luxembourgish", "Macedonian", "Malagasy",
    "Malawian", "Malaysian", "Maldivian", "Malian", "Maltese", "Marshallese",
    "Mauritanian", "Mauritian", "Mexican", "Micronesian", "Moldovan", "Monacan",
    "Mongolian", "Montenegrin", "Moroccan", "Mozambican", "Namibian", "Nauruan",
    "Nepalese", "New Zealander", "Nicaraguan", "Nigerien", "Nigerian", "North Korean",
    "Norwegian", "Omani", "Pakistani", "Palauan", "Palestinian", "Panamanian",
    "Papua New Guinean", "Paraguayan", "Peruvian", "Polish", "Portuguese", "Qatari",
    "Romanian", "Russian", "Rwandan", "Saint Lucian", "Salvadoran", "Samoan",
    "San Marinese", "Sao Tomean", "Saudi", "Senegalese", "Serbian", "Seychellois",
    "Sierra Leonean", "Singaporean", "Slovak", "Slovenian", "Solomon Islander", "Somali",
    "South African", "South Korean", "South Sudanese", "Spanish", "Sri Lankan", "Sudanese",
    "Surinamese", "Swazi", "Swedish", "Swiss", "Syrian", "Taiwanese",
    "Tajik", "Tanzanian", "Thai", "Togolese", "Tongan", "Trinidadian or Tobagonian",
    "Tunisian", "Turkish", "Turkmen", "Tuvaluan", "Ugandan", "Ukrainian",
    "Uruguayan", "Uzbek", "Vanuatuan", "Venezuelan", "Vietnamese", "Vincentian",
    "Yemeni", "Zambian", "Zimbabwean"
]

countries = [
    "Afghanistan", "Albania", "Algeria", "Andorra", "Angola", "Antigua and Barbuda",
    "Argentina", "Armenia", "Australia", "Austria", "Azerbaijan", "Bahamas",
    "Bahrain", "Bangladesh", "Barbados", "Belarus", "Belgium", "Belize",
    "Benin", "Bhutan", "Bolivia", "Bosnia and Herzegovina", "Botswana", "Brazil",
    "Brunei", "Bulgaria", "Burkina Faso", "Burundi", "Cabo Verde", "Cambodia",
    "Cameroon", "Canada", "Central African Republic", "Chad", "Chile", "China",
    "Colombia", "Comoros", "Congo, Democratic Republic of the", "Congo, Republic of the",
    "Costa Rica", "Croatia", "Cuba", "Cyprus", "Czech Republic", "Denmark",
    "Djibouti", "Dominica", "Dominican Republic", "East Timor", "Ecuador", "Egypt",
    "El Salvador", "Equatorial Guinea", "Eritrea", "Estonia", "Eswatini", "Ethiopia",
    "Fiji", "Finland", "France", "Gabon", "Gambia", "Georgia",
    "Germany", "Ghana", "Greece", "Grenada", "Guatemala", "Guinea",
    "Guinea-Bissau", "Guyana", "Haiti", "Honduras", "Hungary", "Iceland",
    "India", "Indonesia", "Iran", "Iraq", "Ireland", "Israel",
    "Italy", "Ivory Coast", "Jamaica", "Japan", "Jordan", "Kazakhstan",
    "Kenya", "Kiribati", "Korea, North", "Korea, South", "Kosovo", "Kuwait",
    "Kyrgyzstan", "Laos", "Latvia", "Lebanon", "Lesotho", "Liberia",
    "Libya", "Liechtenstein", "Lithuania", "Luxembourg", "Madagascar", "Malawi",
    "Malaysia", "Maldives", "Mali", "Malta", "Marshall Islands", "Mauritania",
    "Mauritius", "Mexico", "Micronesia", "Moldova", "Monaco", "Mongolia",
    "Montenegro", "Morocco", "Mozambique", "Myanmar (Burma)", "Namibia", "Nauru",
    "Nepal", "Netherlands", "New Zealand", "Nicaragua", "Niger", "Nigeria",
    "North Macedonia", "Norway", "Oman", "Pakistan", "Palau", "Palestine",
    "Panama", "Papua New Guinea", "Paraguay", "Peru", "Philippines", "Poland",
    "Portugal", "Qatar", "Romania", "Russia", "Rwanda", "Saint Kitts and Nevis",
    "Saint Lucia", "Saint Vincent and the Grenadines", "Samoa", "San Marino", "Sao Tome and Principe", "Saudi Arabia",
    "Senegal", "Serbia", "Seychelles", "Sierra Leone", "Singapore", "Slovakia",
    "Slovenia", "Solomon Islands", "Somalia", "South Africa", "South Sudan", "Spain",
    "Sri Lanka", "Sudan", "Suriname", "Sweden", "Switzerland", "Syria",
    "Taiwan", "Tajikistan", "Tanzania", "Thailand", "Togo", "Tonga",
    "Trinidad and Tobago", "Tunisia", "Turkey", "Turkmenistan", "Tuvalu", "Uganda",
    "Ukraine", "United Arab Emirates", "United Kingdom", "United States", "Uruguay", "Uzbekistan",
    "Vanuatu", "Vatican City", "Venezuela", "Vietnam", "Yemen", "Zambia", "Zimbabwe"
]

languages = [
    "Afrikaans", "Akan", "Albanian", "Amharic", "Arabic", "Armenian", "Assamese", "Aymara",
    "Azerbaijani", "Balochi", "Bamanankan", "Bashkir", "Basque", "Belarusian", "Bengali",
    "Bhojpuri", "Bislama", "Bosnian", "Brahui", "Bulgarian", "Burmese", "Cantonese",
    "Catalan", "Cebuano", "Chechen", "Cherokee", "Croatian", "Czech", "Danish", "Dari",
    "Dholuo", "Dutch", "Dzongkha", "English", "Esperanto", "Estonian", "Ewe", "Finnish",
    "Flemish", "French", "Fulfulde", "Gaelic", "Galician",
    "Ganda", "Georgian", "German", "Gikuyu", "Greek", "Guarani", "Gujarati", "Haitian Creole",
    "Hausa", "Hawaiian", "Hebrew", "Hindi", "Hmong", "Hungarian", "Icelandic", "Igbo",
    "Ilocano", "Indonesian", "Inuit/Inuktitut", "Italian", "Japanese", "Jarai", "Javanese",
    "Kiche", "Kannada", "Kashmiri", "Kazakh", "Khmer", "Kinyarwanda", "Kirundi", "Korean",
    "Kurdish", "Kyrgyz", "Lao", "Latvian", "Lingala", "Lithuanian", "Luganda", "Luxembourgish",
    "Macedonian", "Maithili", "Malagasy", "Malay", "Malayalam", "Maltese", "Māori",
    "Marathi", "Mende", "Mongolian", "Nahuatl", "Nauruan", "Navajo", "Nepali", "Norwegian",
    "Ojibwa", "Oriya", "Oromo", "Pashto", "Persian", "Polish", "Portuguese",
    "Punjabi", "Quechua", "Romani", "Romanian", "Russian", "Rwanda", "Samoan", "Sango",
    "Sanskrit", "Serbian", "Shona", "Sindhi", "Sinhala", "Slovak", "Slovene", "Somali",
    "Spanish", "Swahili", "Swedish", "Tajiki", "Tamil", "Tatar", "Telugu", "Thai", "Tibetic",
    "Tigrigna", "Tok Pisin", "Turkish", "Turkmen", "Twi", "Uighur", "Ukrainian", "Urdu",
    "Uzbek", "Vietnamese", "Warlpiri", "Welsh", "Wolof", "Xhosa", "Yoruba", "Yucatec",
    "Zapotec", "Zulu"
]

nationalities_pattern = '|'.join(nationalities)
countries_pattern = '|'.join(countries)
languages_pattern = '|'.join(languages)
patterns = [rf"\bI'm (not )?({nationalities_pattern})\b",
            rf"\bI’m (not )?({nationalities_pattern})\b",
            rf"\b being ?({nationalities_pattern})\b",
            rf"\bI am (not )?({nationalities_pattern})\b",
            rf"\bI'm (not )?from ({countries_pattern})\b",
            rf"\bI’m (not )?from ({countries_pattern})\b",
            rf"\bI am (not )?from ({countries_pattern})\b",
            rf"\blive in ({countries_pattern})\b",
            rf"\bnative language is ({languages_pattern})\b",
            rf"\bnative language isn't ({languages_pattern})\b",
            rf"\bnative language isn’t ({languages_pattern})\b",
            rf"\bnative language is not ({languages_pattern})\b",
            rf"\b({languages_pattern}) is my native language\b",
            rf"\b({languages_pattern}) isn't my native language\b",
            rf"\b({languages_pattern}) isn’t my native language\b",
            rf"\b({languages_pattern}) is not my native language\b",
            rf"\bspeak ({languages_pattern})\b",
            rf"\bmain language is ({languages_pattern})\b",
            rf"\bmain language isn't ({languages_pattern})\b",
            rf"\bmain language isn’t ({languages_pattern})\b",
            rf"\bmain language is not ({languages_pattern})\b",
            rf"\b({languages_pattern}) is my main language\b",
            rf"\b({languages_pattern}) isn't my main language\b",
            rf"\b({languages_pattern}) isn’t my main language\b",
            rf"\b({languages_pattern}) is not my main language\b"
            ]
print(patterns)

compiled_patterns = [re.compile(pattern, re.IGNORECASE) for pattern in patterns]
print(compiled_patterns)

["\\bI'm (not )?(Afghan|Albanian|Algerian|American|Andorran|Angolan|Antiguan|Argentine|Armenian|Australian|Austrian|Azerbaijani|Bahamian|Bahraini|Bangladeshi|Barbadian|Barbudan|Batswana|Belarusian|Belgian|Belizean|Beninese|Bhutanese|Bolivian|Bosnian|Brazilian|British|Bruneian|Bulgarian|Burkinabé|Burmese|Burundian|Cabo Verdean|Cambodian|Cameroonian|Canadian|Central African|Chadian|Chilean|Chinese|Colombian|Comoran|Congolese|Costa Rican|Croatian|Cuban|Cypriot|Czech|Danish|Djiboutian|Dominican|Dutch|East Timorese|Ecuadorean|Egyptian|Emirati|Equatorial Guinean|Eritrean|Estonian|Ethiopian|Fijian|Finnish|French|Gabonese|Gambian|Georgian|German|Ghanaian|Greek|Grenadian|Guatemalan|Guinean|Guinea-Bissauan|Guyanese|Haitian|Honduran|Hungarian|Icelandic|Indian|Indonesian|Iranian|Iraqi|Irish|Israeli|Italian|Ivorian|Jamaican|Japanese|Jordanian|Kazakh|Kenyan|Kiribati|Kittian and Nevisian|Kuwaiti|Kyrgyz|Laotian|Latvian|Lebanese|Liberian|Libyan|Liechtensteiner|Lithuanian|Luxembourgish|Macedonian|Malaga

In [ ]:
import re
import pandas as pd
import csv

# Read the input CSV file from Google Drive
input_csv_path = 'path'
df = pd.read_csv(input_csv_path)

# Path to the output CSV file where matches will be saved
output_csv_path = 'path'

# Open the output CSV file in append mode
with open(output_csv_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Write the header to the output CSV
    header = list(df.columns) + ['matched_patterns']
    writer.writerow(header)

    # Iterate through each row, checking the content in columns E and F (index 4 and 5)
    for index, row in df.iterrows():
        if row['username'] not in values_set_final:
          pass
        else:
          matches = []
          title = str(row['title']).lower() if pd.notna(row['title']) else ''
          body = str(row['body']).lower() if pd.notna(row['body']) else ''

          for pattern in compiled_patterns:
              # Check if the pattern is found in either the lowercase title or body
              if pattern.search(title) or pattern.search(body):
                  matches.append(pattern.pattern)

          # If there were matches, save the row immediately to the output CSV
          if matches:
              matched_row = list(row) + ['; '.join(matches)]
              writer.writerow(matched_row)

print(f"Filtered data is done")

Filtered data is done


In [ ]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 2.9 MB/s eta 0:00:00


In [ ]:
import csv
from collections import Counter
from google.colab import files
uploaded = files.upload()

subreddits_set = set()
subreddits = []
with open('subreddits_sz_final - subreddits_sz_final (1).csv', 'r') as file:
  reader = csv.reader(file)
  for row in reader:
    if row[1] == 'X':
      subreddits_set.add(row[0])
print(subreddits_set)

redditors_set = set()
import re
import emoji

# Function to remove links from text
def remove_links(text):
    return re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Function to remove emojis from text
def remove_emojis(text):
    return emoji.replace_emoji(text, replace='')  # Replace emojis with empty string

subreddits = []
with open('path', 'r') as infile, open('path', 'w', newline='') as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile)

    # Write header to output file if the CSV contains a header
    header = next(reader)
    writer.writerow(header)  # Adjust this if there is no header


    # Iterate through the rows and collect subreddit names from the seventh column
    for row in reader:
      if row[6] in subreddits_set:
        pass
      else:
        row[4] = remove_emojis(remove_links(row[4]))
        row[5] = remove_emojis(remove_links(row[5]))
        combined_row = row[4] + row[5]
        if len(combined_row.split()) >= 5:
          writer.writerow(row)
          subreddits.append(row[6])  # The seventh column (index 6)
          redditors_set.add(row[0])
        else:
          pass

# Count the occurrences of each subreddit
subreddit_counter = Counter(subreddits)

# Get the top 10 most common subreddits
top_10_subreddits = subreddit_counter.most_common(10)

# Print the top 10 subreddits
for subreddit, count in top_10_subreddits:
    print(f'{subreddit}: {count} occurrences')

CONTROLS Reddit and Regex Search

In [ ]:
users_list = ["LIST_OF_SZ_USERS"]

print(len(users_list))

In [ ]:
top_10_subreddits = ['AskReddit', '2007scape', 'teenagers', 'GenX', 'NoStupidQuestions', 'pics', 'Tulpas', 'AskWomen', 'DodoCodeCommunity', 'RandomActsOfGaming']
# From the SZ group

In [ ]:
# List of common phrases
phrases = [
    r"i have",
    r"i've",
    r"ive",
    r"i'm",
    r"suffering from",
    r"suffer from",
    r"being",
    r"diagnosis of",
    r"diagnosed with",
    r"diagnosed me with",
    r"diagnostic of",
    r"diagnosed",
    r"hospitalized with",
    r"hospital with",
    r"my"
]

# List of mental disorders
disorders = [
    # Intellectual Developmental Disorder (Intellectual Disability) V
    'intellectual disability', 'intellectual developmental', 'idd', 'mental retardation', 'developmental delay', 'developmental disabilit', 'hypertelorism', 'facies', 'snijders', 'child development deviation', 'development deviation', 'mental deficiency',

    # Language Disorder V
    'language', 'speech delay', 'expressive language', 'receptive language', 'aphasia', 'Acquired Language Disorder', 'language impairment', 'speech disorder', 'specific language disorder', 'sld', 'developmental disorder', 'language delay', 'semantic-pragmatic', 'semantic pragmatic', 'central auditory', 'auditory central', 'speech or language developmental',

    # Speech Sound Disorder V
    'speech sound', 'articulation', 'phonological', 'apraxia', 'dysarthria',

    # Social (Pragmatic) Communication Disorder V
    'social communication', 'pragmatic', 'social language',

    # Unspecified Communication Disorder V
    'communication', 'unspecified communication', 'speech and language', 'childhood communication', 'acquired communication', 'communicative', 'developmental communication', 'neurogenic', 'hearing disorder',

    # Autism Spectrum Disorder V
    'autism', 'asd', 'asperger', 'high-functioning autism', 'pervasive developmental disorder', 'autie', 'autistic', 'kenner', 'Cerebroatrophic Hyperammonemia', 'rett', 'akinetic',

    # Bipolar I V
    'bipolar', 'manic', 'mania',

    # Bipolar II V
    'hypomania',

    # Cyclothymia V
    'cyclothymi',

    # Factitious Disorder V
    'factitious', 'genser syndrome', 'pseudodementia', 'pseudopsychosis',

    # Alzheimer/Dementia V
    'alzheimer', 'dementia', 'cognitive decline', 'amentia', 'senile paranoid dementia', 'familial dementia', 'early-onset dementia', 'acute confuse senile', 'early onset alzheimer', 'familiar alzheimer disease', 'focal onset alzheimer', 'late onset alzheimer', 'presenile alzheimer', 'primary senile degenerative dementia', 'senile dementia',

    # Brain Injury V
    'brain injury', 'traumatic brain injury', 'tbi', 'concussion', 'head injury', 'focal brain inj', 'acute brain inj', 'brain laceration', 'chronic brain inj',

    # Lewy Body Disease V
    'lewy body disease', 'lewy body dementia', 'lbd', 'cortical lewy body disease', 'diffuse lewy body', 'lewy bod',

    # Frontotemporal Lobar Degeneration V
    'frontotemporal lobar degeneration', 'ftld', 'frontotemporal dementia', 'pick\'s disease', 'ddpac', 'familial pick\'s disease', 'hddd', 'semantic dementia', 'Wilhelmsen-Lynch Disease', 'Hereditary Dysphasic Disinhibition Dementia', 'Multiple System Tauopathy with Presenile Dementia',

    # Parkinson's Disease V
    'parkinson', 'parkinsonism', 'parkinsonian syndrome', 'idiopathic parkinson', 'paralysis agitans', 'primary parkinsonism',

    # Huntington's Disease V
    'huntington', 'hd', 'Akinetic-Rigid Variant of Huntington', 'chorea', 'Chronic Progressive Hereditary Chorea', 'Juvenile Huntington', 'Juvenile-Onset Huntington', 'Late-Onset Huntington Disease', 'Late Onset Huntington Disease', 'Juvenile Onset Huntington', 'Progressive Chorea',

    # Specific Learning Disorder (e.g., Dyslexia) V
    'specific learning', 'learning', 'dyslexia', 'dysgraphia', 'dyscalculia', 'sld',

    # Borderline Personality Disorder V
    'borderline', 'bpd', 'bp-d'
]

patterns = [fr"{phrase}\s*({disorder})" for phrase in phrases for disorder in disorders]

compiled_patterns_disorders = [re.compile(pattern, re.IGNORECASE) for pattern in patterns]
print(compiled_patterns_disorders)

In [ ]:
# List of common phrases
phrases = [
    r"i have",
    r"i've",
    r"ive",
    r"i'm",
    r"suffering from",
    r"suffer from",
    r"being",
    r"diagnosis of",
    r"diagnosed with",
    r"diagnosed me with",
    r"diagnostic of",
    r"diagnosed",
    r"hospitalized with",
    r"hospital with",
    r"my"
]

# List of mental disorders
disorders = [
    "schzo",
    "schizopheni",
    "scizophr",
    "schophreni",
    "schizopr",
    "shizophrenia",
    "schizphrenia",
    "schzophrenia",
    "schizoprenia",
    "scizophrenia",
    "schizohrenia",
    "schzophreni",
    "schzophrnia",
    "schizohrnia",
    "schizophenia",
    "schizorphenia",
    "schizoprhenia",
    "schzophrena",
    "schizorena",
    "scizophrnia"
]

patterns = [fr"{phrase}\s*({disorder})" for phrase in phrases for disorder in disorders]

compiled_patterns_schizophrenia = [re.compile(pattern, re.IGNORECASE) for pattern in patterns]
print(compiled_patterns_schizophrenia)

In [ ]:
nationalities = [
    "Afghan", "Albanian", "Algerian", "American", "Andorran", "Angolan",
    "Antiguan", "Argentine", "Armenian", "Australian", "Austrian", "Azerbaijani",
    "Bahamian", "Bahraini", "Bangladeshi", "Barbadian", "Barbudan", "Batswana",
    "Belarusian", "Belgian", "Belizean", "Beninese", "Bhutanese", "Bolivian",
    "Bosnian", "Brazilian", "British", "Bruneian", "Bulgarian", "Burkinabé",
    "Burmese", "Burundian", "Cabo Verdean", "Cambodian", "Cameroonian", "Canadian",
    "Central African", "Chadian", "Chilean", "Chinese", "Colombian", "Comoran",
    "Congolese", "Costa Rican",
    "Croatian", "Cuban", "Cypriot", "Czech", "Danish", "Djiboutian", "Dominican",
    "Dutch", "East Timorese", "Ecuadorean", "Egyptian", "Emirati", "Equatorial Guinean",
    "Eritrean", "Estonian", "Ethiopian", "Fijian", "Finnish", "French",
    "Gabonese", "Gambian", "Georgian", "German", "Ghanaian", "Greek",
    "Grenadian", "Guatemalan", "Guinean", "Guinea-Bissauan", "Guyanese", "Haitian",
    "Honduran", "Hungarian", "Icelandic", "Indian", "Indonesian", "Iranian",
    "Iraqi", "Irish", "Israeli", "Italian", "Ivorian", "Jamaican",
    "Japanese", "Jordanian", "Kazakh", "Kenyan", "Kiribati", "Kittian and Nevisian",
    "Kuwaiti", "Kyrgyz", "Laotian", "Latvian", "Lebanese", "Liberian",
    "Libyan", "Liechtensteiner", "Lithuanian", "Luxembourgish", "Macedonian", "Malagasy",
    "Malawian", "Malaysian", "Maldivian", "Malian", "Maltese", "Marshallese",
    "Mauritanian", "Mauritian", "Mexican", "Micronesian", "Moldovan", "Monacan",
    "Mongolian", "Montenegrin", "Moroccan", "Mozambican", "Namibian", "Nauruan",
    "Nepalese", "New Zealander", "Nicaraguan", "Nigerien", "Nigerian", "North Korean",
    "Norwegian", "Omani", "Pakistani", "Palauan", "Palestinian", "Panamanian",
    "Papua New Guinean", "Paraguayan", "Peruvian", "Polish", "Portuguese", "Qatari",
    "Romanian", "Russian", "Rwandan", "Saint Lucian", "Salvadoran", "Samoan",
    "San Marinese", "Sao Tomean", "Saudi", "Senegalese", "Serbian", "Seychellois",
    "Sierra Leonean", "Singaporean", "Slovak", "Slovenian", "Solomon Islander", "Somali",
    "South African", "South Korean", "South Sudanese", "Spanish", "Sri Lankan", "Sudanese",
    "Surinamese", "Swazi", "Swedish", "Swiss", "Syrian", "Taiwanese",
    "Tajik", "Tanzanian", "Thai", "Togolese", "Tongan", "Trinidadian or Tobagonian",
    "Tunisian", "Turkish", "Turkmen", "Tuvaluan", "Ugandan", "Ukrainian",
    "Uruguayan", "Uzbek", "Vanuatuan", "Venezuelan", "Vietnamese", "Vincentian",
    "Yemeni", "Zambian", "Zimbabwean"
]

countries = [
    "Afghanistan", "Albania", "Algeria", "Andorra", "Angola", "Antigua and Barbuda",
    "Argentina", "Armenia", "Australia", "Austria", "Azerbaijan", "Bahamas",
    "Bahrain", "Bangladesh", "Barbados", "Belarus", "Belgium", "Belize",
    "Benin", "Bhutan", "Bolivia", "Bosnia and Herzegovina", "Botswana", "Brazil",
    "Brunei", "Bulgaria", "Burkina Faso", "Burundi", "Cabo Verde", "Cambodia",
    "Cameroon", "Canada", "Central African Republic", "Chad", "Chile", "China",
    "Colombia", "Comoros", "Congo, Democratic Republic of the", "Congo, Republic of the",
    "Costa Rica", "Croatia", "Cuba", "Cyprus", "Czech Republic", "Denmark",
    "Djibouti", "Dominica", "Dominican Republic", "East Timor", "Ecuador", "Egypt",
    "El Salvador", "Equatorial Guinea", "Eritrea", "Estonia", "Eswatini", "Ethiopia",
    "Fiji", "Finland", "France", "Gabon", "Gambia", "Georgia",
    "Germany", "Ghana", "Greece", "Grenada", "Guatemala", "Guinea",
    "Guinea-Bissau", "Guyana", "Haiti", "Honduras", "Hungary", "Iceland",
    "India", "Indonesia", "Iran", "Iraq", "Ireland", "Israel",
    "Italy", "Ivory Coast", "Jamaica", "Japan", "Jordan", "Kazakhstan",
    "Kenya", "Kiribati", "Korea, North", "Korea, South", "Kosovo", "Kuwait",
    "Kyrgyzstan", "Laos", "Latvia", "Lebanon", "Lesotho", "Liberia",
    "Libya", "Liechtenstein", "Lithuania", "Luxembourg", "Madagascar", "Malawi",
    "Malaysia", "Maldives", "Mali", "Malta", "Marshall Islands", "Mauritania",
    "Mauritius", "Mexico", "Micronesia", "Moldova", "Monaco", "Mongolia",
    "Montenegro", "Morocco", "Mozambique", "Myanmar (Burma)", "Namibia", "Nauru",
    "Nepal", "Netherlands", "New Zealand", "Nicaragua", "Niger", "Nigeria",
    "North Macedonia", "Norway", "Oman", "Pakistan", "Palau", "Palestine",
    "Panama", "Papua New Guinea", "Paraguay", "Peru", "Philippines", "Poland",
    "Portugal", "Qatar", "Romania", "Russia", "Rwanda", "Saint Kitts and Nevis",
    "Saint Lucia", "Saint Vincent and the Grenadines", "Samoa", "San Marino", "Sao Tome and Principe", "Saudi Arabia",
    "Senegal", "Serbia", "Seychelles", "Sierra Leone", "Singapore", "Slovakia",
    "Slovenia", "Solomon Islands", "Somalia", "South Africa", "South Sudan", "Spain",
    "Sri Lanka", "Sudan", "Suriname", "Sweden", "Switzerland", "Syria",
    "Taiwan", "Tajikistan", "Tanzania", "Thailand", "Togo", "Tonga",
    "Trinidad and Tobago", "Tunisia", "Turkey", "Turkmenistan", "Tuvalu", "Uganda",
    "Ukraine", "United Arab Emirates", "United Kingdom", "United States", "Uruguay", "Uzbekistan",
    "Vanuatu", "Vatican City", "Venezuela", "Vietnam", "Yemen", "Zambia", "Zimbabwe"
]

languages = [
    "Afrikaans", "Akan", "Albanian", "Amharic", "Arabic", "Armenian", "Assamese", "Aymara",
    "Azerbaijani", "Balochi", "Bamanankan", "Bashkir", "Basque", "Belarusian", "Bengali",
    "Bhojpuri", "Bislama", "Bosnian", "Brahui", "Bulgarian", "Burmese", "Cantonese",
    "Catalan", "Cebuano", "Chechen", "Cherokee", "Croatian", "Czech", "Danish", "Dari",
    "Dholuo", "Dutch", "Dzongkha", "English", "Esperanto", "Estonian", "Ewe", "Finnish",
    "Flemish", "French", "Fulfulde", "Gaelic", "Galician",
    "Ganda", "Georgian", "German", "Gikuyu", "Greek", "Guarani", "Gujarati", "Haitian Creole",
    "Hausa", "Hawaiian", "Hebrew", "Hindi", "Hmong", "Hungarian", "Icelandic", "Igbo",
    "Ilocano", "Indonesian", "Inuit/Inuktitut", "Italian", "Japanese", "Jarai", "Javanese",
    "Kiche", "Kannada", "Kashmiri", "Kazakh", "Khmer", "Kinyarwanda", "Kirundi", "Korean",
    "Kurdish", "Kyrgyz", "Lao", "Latvian", "Lingala", "Lithuanian", "Luganda", "Luxembourgish",
    "Macedonian", "Maithili", "Malagasy", "Malay", "Malayalam", "Maltese", "Māori",
    "Marathi", "Mende", "Mongolian", "Nahuatl", "Nauruan", "Navajo", "Nepali", "Norwegian",
    "Ojibwa", "Oriya", "Oromo", "Pashto", "Persian", "Polish", "Portuguese",
    "Punjabi", "Quechua", "Romani", "Romanian", "Russian", "Rwanda", "Samoan", "Sango",
    "Sanskrit", "Serbian", "Shona", "Sindhi", "Sinhala", "Slovak", "Slovene", "Somali",
    "Spanish", "Swahili", "Swedish", "Tajiki", "Tamil", "Tatar", "Telugu", "Thai", "Tibetic",
    "Tigrigna", "Tok Pisin", "Turkish", "Turkmen", "Twi", "Uighur", "Ukrainian", "Urdu",
    "Uzbek", "Vietnamese", "Warlpiri", "Welsh", "Wolof", "Xhosa", "Yoruba", "Yucatec",
    "Zapotec", "Zulu"
]

nationalities_pattern = '|'.join(nationalities)
countries_pattern = '|'.join(countries)
languages_pattern = '|'.join(languages)
patterns = [rf"\bI'm (not )?({nationalities_pattern})\b",
            rf"\bI’m (not )?({nationalities_pattern})\b",
            rf"\b being ?({nationalities_pattern})\b",
            rf"\bI am (not )?({nationalities_pattern})\b",
            rf"\bI'm (not )?from ({countries_pattern})\b",
            rf"\bI’m (not )?from ({countries_pattern})\b",
            rf"\bI am (not )?from ({countries_pattern})\b",
            rf"\blive in ({countries_pattern})\b",
            rf"\bnative language is ({languages_pattern})\b",
            rf"\bnative language isn't ({languages_pattern})\b",
            rf"\bnative language isn’t ({languages_pattern})\b",
            rf"\bnative language is not ({languages_pattern})\b",
            rf"\b({languages_pattern}) is my native language\b",
            rf"\b({languages_pattern}) isn't my native language\b",
            rf"\b({languages_pattern}) isn’t my native language\b",
            rf"\b({languages_pattern}) is not my native language\b",
            rf"\bspeak ({languages_pattern})\b",
            rf"\bmain language is ({languages_pattern})\b",
            rf"\bmain language isn't ({languages_pattern})\b",
            rf"\bmain language isn’t ({languages_pattern})\b",
            rf"\bmain language is not ({languages_pattern})\b",
            rf"\b({languages_pattern}) is my main language\b",
            rf"\b({languages_pattern}) isn't my main language\b",
            rf"\b({languages_pattern}) isn’t my main language\b",
            rf"\b({languages_pattern}) is not my main language\b"
            ]
print(patterns)

compiled_patterns_nationalities = [re.compile(pattern, re.IGNORECASE) for pattern in patterns]
print(compiled_patterns_nationalities)

In [ ]:
controls_list = set()  # Set to track already processed users
max_users = 100  # Limit the number of users to fetch per subreddit

# Function to save user posts and comments to CSV
def save_posts_to_csv(username, posts_data):
    with open(r"path", mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        for data in posts_data:
            writer.writerow([username, data['type'], data['post_id'], data['created_utc'],
                             data['title'], data['body'], data['subreddit']])

In [ ]:
users_list = set(users_list)

In [ ]:
# Iterate through each subreddit in the list
for subreddit_name in top_10_subreddits:
    subreddit = reddit.subreddit(subreddit_name)
    subreddit_users = set()  # Track users found in each subreddit

    try:
        # Get the top submissions from the subreddit (time_filter='all' for all time)
        top_submissions = subreddit.top(limit=None, time_filter='all')

        for submission in top_submissions:
            if len(subreddit_users) >= max_users:
                break  # Stop if we reached 100 unique users

            try:
                redditor_name = submission.author.name if submission.author else None
                print(redditor_name)

                # If the user is valid and not already processed
                if redditor_name and redditor_name not in users_list:
                    subreddit_users.add(redditor_name)
                    users_list.add(redditor_name)
                    print(f"Fetching posts for user: {redditor_name}")

                    # Fetch the user's posts and comments
                    redditor = reddit.redditor(redditor_name)
                    posts_data = []

                    # Get posts and comments
                    for item in redditor.new(limit=None):
                        created_utc = datetime.utcfromtimestamp(item.created_utc).strftime('%Y-%m-%d %H:%M')
                        post_data = {
                            'type': 'post' if isinstance(item, praw.models.Submission) else 'comment',
                            'post_id': item.id,
                            'created_utc': created_utc,
                            'title': item.title if isinstance(item, praw.models.Submission) else '',
                            'body': item.selftext if isinstance(item, praw.models.Submission) else item.body,
                            'subreddit': item.subreddit.display_name
                        }
                        posts_data.append(post_data)

                    # Append the user's posts and comments to the CSV
                    save_posts_to_csv(redditor_name, posts_data)

            except Exception as e:
                # Catch any other issues with individual users (API issues, etc.)
                print(f"Error fetching data for user {redditor_name}: {e}")
                continue

        if len(subreddit_users) < max_users:
            print(f"Subreddit {subreddit_name} doesn't have enough unique users, skipping.")
            continue  # Skip to next subreddit if we didn't find enough users

        print(f"Finished fetching users from subreddit: {subreddit_name}")

    except Exception as e:
        # Catch issues with fetching subreddit data or API limits
        print(f"Error fetching data from subreddit {subreddit_name}: {e}")
        time.sleep(60)  # Wait before continuing to avoid hitting rate limits
        continue

print("Completed fetching users and their posts.")

In [ ]:
# Paths
input_csv_path = "drive/My Drive/controls_top_10_subreddits_submissions_only.csv"
output_posts_csv_path = "drive/My Drive/matched_posts_with_patterns.csv"
output_users_csv_path = "drive/My Drive/matched_users_and_patterns.csv"

In [ ]:
# Lists to track matched posts and users
matched_posts = []
matched_users = {}

In [ ]:
# Open the input CSV file
with open(input_csv_path, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)

    # Iterate through each row in the CSV file (each post or comment)
    for row in reader:
        title = row['title']
        body = row['body']
        username = row['username']

        # List to keep track of which patterns matched for this post/comment
        matched_patterns = []

        # Check title and body for matches
        for pattern in compiled_patterns_schizophrenia:
            if pattern.search(title) or pattern.search(body):
                matched_patterns.append('schizophrenia')

        for pattern in compiled_patterns_disorders:
            if pattern.search(title) or pattern.search(body):
                matched_patterns.append('disorders')
        for pattern in compiled_patterns_nationalities:
            if pattern.search(title) or pattern.search(body):
                matched_patterns.append('nationalities')

        # If any pattern matched, append the row to matched_posts and update matched_users
        if matched_patterns:
            row['matched_patterns'] = ', '.join(matched_patterns)
            matched_posts.append(row)

            if username in matched_users:
                matched_users[username].update(matched_patterns)
            else:
                matched_users[username] = set(matched_patterns)

In [ ]:
# Write matched posts with found patterns to a new CSV
with open(output_posts_csv_path, mode='w', newline='', encoding='utf-8') as file:
    fieldnames = ['username', 'type', 'post_id', 'created_ufc', 'title', 'body', 'subreddit', 'matched_patterns']
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    writer.writeheader()
    for post in matched_posts:
        writer.writerow(post)

# Write usernames and matched patterns to a new CSV
with open(output_users_csv_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    writer.writerow(['username', 'matched_patterns'])
    for username, patterns in matched_users.items():
        writer.writerow([username, ', '.join(patterns)])

print("Process completed. Two new CSV files have been created.")

In [ ]:
input_csv_path = "drive/My Drive/controls_top_10_subreddits_submissions_only.csv"
output_users_csv_path = "drive/My Drive/matched_users_and_patterns.csv"

# Function to get the number of unique users from a CSV file
def count_unique_users(csv_path, username_column_name):
    unique_users = set()

    with open(csv_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)

        for row in reader:
            username = row[username_column_name]
            unique_users.add(username)

    return len(unique_users), unique_users

# Count unique users in the input CSV
num_users_in_input_csv, input_unique_users = count_unique_users(input_csv_path, 'username')
print(f"Number of unique users in '{input_csv_path}': {num_users_in_input_csv}")

# Count unique users in the output CSV
num_users_in_output_csv, output_unique_users = count_unique_users(output_users_csv_path, 'username')
print(f"Number of unique users in '{output_users_csv_path}': {num_users_in_output_csv}")
difference_num_users = num_users_in_input_csv - num_users_in_output_csv

print(f'Difference: {difference_num_users}')

In [ ]:
print("List of users that were not tagged by any filtering search:")
for user in input_unique_users - output_unique_users:
    print(user)

In [ ]:
controls_list_final = input_unique_users - output_unique_users
print(controls_list_final)

In [ ]:
input_csv_path = "drive/My Drive/controls_top_10_subreddits_submissions_only.csv"
output_csv_path = "drive/My Drive/controls_finals_posts.csv"

# Open the output CSV file in write mode
with open(output_csv_path, mode='w', newline='', encoding='utf-8') as output_file:
    # Open the input CSV file
    with open(input_csv_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        # Create a writer object to write the rows into the output file
        writer = csv.DictWriter(output_file, fieldnames=reader.fieldnames)

        # Write the header from the input file to the output file
        writer.writeheader()

        # Iterate through each row in the input CSV file (each post or comment)
        for row in reader:
            if row['username'] in controls_list_final:
                # Write the row to the output file if the username is in controls_list_final
                writer.writerow(row)

print("Process completed. Rows have been written to the output file.")

All resulting posts collected from controls also went through emoji and link removals and only those with 5 words or more were considered. Finally, SZ and control posts went through unicode standardization:

In [ ]:
!pip install unicodedata
import unicodedata

post_count = 0
redditors_set = set()
subreddits = []
post_count_before = 0

with open("path", 'r') as file:
  reader = csv.reader(file)
  next(reader)
  for row in reader:
    post_count_before +=1

with open("path", 'r') as infile, open("path", 'w', newline='') as outfile:
  reader = csv.reader(infile)
  writer = csv.writer(outfile)

    # Write header to output file if the CSV contains a header
  header = next(reader)
  writer.writerow(header)
  for row in reader:
    row = [unicodedata.normalize('NFKD', cell).encode('ascii', 'ignore').decode('utf-8') for cell in row]

    combined_row = row[4] + row[5]
    if len(combined_row.split()) >= 5:
      writer.writerow(row)
      subreddits.append(row[6])
      redditors_set.add(row[0])
      post_count +=1
    else:
      pass

# Count the occurrences of each subreddit
subreddit_counter = Counter(subreddits)

# Get the top 10 most common subreddits
top_10_subreddits = subreddit_counter.most_common(10)

print(f'Lista de usuários depois dos filtros: {len(redditors_set)}')
print(redditors_set)

# Print the top 10 subreddits
for subreddit, count in top_10_subreddits:
    print(f'{subreddit}: {count} occurrences')
